# Introduction to Aurite Agents: Your First AI Agent

Welcome to the Aurite Agents framework! This tutorial is the first in a series designed to guide you through building and running your own AI agents. 

**Goal:** By the end of this notebook, you will have configured and run a simple AI agent that can answer a question using a powerful Large Language Model (LLM).

**What is Aurite Agents?** It's a Python framework that provides the building blocks to create, manage, and deploy AI agents. It handles the complex parts of interacting with LLMs and external tools, so you can focus on defining what your agent does.

**Prerequisites:**

*   Python 3.8+ installed (Google Colab uses Python 3.11)

*   An API key from an LLM provider (we'll use OpenAI in this example).

## 1. Installing the aurite Python package

The Aurite Agents framework comes in a number of forms, the primary way to use the framework is through the aurite python package.

Let's install it.

In [1]:
%pip install aurite


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 1. Setup and Imports

First, let's install the required packages and import the necessary classes from the `aurite` library and other standard Python modules to prepare for the next steps.

In [ ]:
from aurite import Aurite

# This is the main entry point for your Aurite project.
# You can use this class to register and execute LLMs, MCP Servers, Agents, Simple Workflows, and Custom Workflows.
aurite = Aurite()

# Initialization is where we start up the MCP Tool servers, along with loading your project json file.
await aurite.initialize()


  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached mcp-1.9.3-py3-none-any.whl.metadata (28 kB)
  Using cached sqlalchemy-2.0.41-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
  Using cached anthropic-0.52.2-py3-none-any.whl.metadata (25 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached google_genai-1.19.0-py3-none-any.whl.metadata (35 kB)
  Using cached greenlet-3.2.3-cp311-cp311-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (4.1 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached iniconfig-2.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached jiter-0.8.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.2 kB)
  Using cached mem0ai-0.1.104-py3-none-any.whl.metadata (7.4 kB)
  Using cached colorlog-6

## 2. Configure Your API Key

To use a Large Language Model like OpenAI's GPT-4, you need an API key. For security, it's best practice to set this as an environment variable rather than writing it directly in your code.

**Action:** Replace `"sk-..."` with your actual OpenAI API key in the cell below.

In [ ]:
# Set the environment variable for the OpenAI API key
# Make sure to replace "sk-..." with your real key
os.environ["OPENAI_API_KEY"] = "sk-..."

# Let's check if the key is set
if not os.getenv("OPENAI_API_KEY") or os.getenv("OPENAI_API_KEY") == "sk-...":
    logger.warning("OpenAI API key is not set. Please add it to proceed.")
else:
    logger.info("OpenAI API key configured successfully.")

## 3. The Building Blocks: Components

The Aurite framework is built around components. Think of them as configurable building blocks. For our first agent, we need two components:

1.  **LLM Component:** This represents the language model that will act as the agent's "brain."

2.  **Agent Component:** This defines the agent itself, including its purpose (via a system prompt) and which LLM it should use.

### Step 3.1: Configure the LLM

We use the `LLMConfig` class to define our LLM. We give it a unique ID, specify the provider and model, and tell it which environment variable holds the API key.

In [ ]:
llm_config = LLMConfig(
    llm_id="openai_gpt4_turbo", # A unique name for this configuration
    provider="openai",
    model_name="gpt-4-turbo-preview",
    api_key_env_var="OPENAI_API_KEY", # The env var we just set
)

print("LLMConfig created:", llm_config.llm_id)

### Step 3.2: Configure the Agent

Next, we use the `AgentConfig` class. We give our agent a name, a `system_prompt` that tells it how to behave, and link it to the LLM configuration we just created using the `llm_config_id`.

In [ ]:
agent_config = AgentConfig(
    name="My First Agent", # A unique name for our agent
    system_prompt="You are a helpful assistant. Your job is to answer the user's question clearly and concisely.",
    llm_config_id="openai_gpt4_turbo", # Connects the agent to our LLM
)

print("AgentConfig created:", agent_config.name)

## 4. Initialize Aurite and Register Components

Now we bring everything together. We create an instance of the main `Aurite` class. This object will manage all our components. We then `initialize` it and `register` the LLM and Agent configurations we defined.

In [ ]:
async def setup_aurite():
    # Initialize the main Aurite application object
    aurite = Aurite()
    await aurite.initialize()

    # Register our LLM and Agent configurations
    await aurite.register_llm_config(llm_config)
    await aurite.register_agent(agent_config)

    print("Aurite initialized and components registered.")
    return aurite

# In a notebook, we can run the async setup like this:
aurite_app = await setup_aurite()

## 5. Run the Agent!

With everything set up, it's time for the fun part. We can now ask our agent a question. We'll use the `run_agent_stream` method from the `execution` facade. This method runs the agent and streams the response back token by token, just like ChatGPT.

In [ ]:
async def run_agent():
    user_query = "What is the mission of the Aurite project?"
    print(colored(f"\n--- Running Agent with Query: '{user_query}' ---", "yellow", attrs=["bold"]))

    # Run the agent with the user's query
    agent_result = await aurite_app.execution.run_agent_stream(
        agent_name="My First Agent",
        user_message=user_query
    )

    response_text = agent_result.primary_text
    print(colored(f"\nAgent's final response: {response_text}", "cyan", attrs=["bold"]))

    # Don't forget to shut down the application to clean up resources
    await aurite_app.shutdown()
    print("\nAurite shutdown complete.")

# Run the agent
await run_agent()

## Congratulations!

You have successfully built and run your first AI agent using the Aurite framework. 

**Recap:**

- You learned how to configure `LLMConfig` and `AgentConfig`.

- You initialized the `Aurite` application and registered your components.

- You ran the agent with a query and received a response.

In the next tutorial, we will explore one of the most powerful features of Aurite: giving agents **tools** to interact with the outside world using the **Model Context Protocol (MCP)**.